#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import pickle

from src.models import model_schemata as schema
from src.models import build_model as build
from src.models import parameter_learning as lfi
from src.models import inference as inf
from src.visualization import visualize as viz


In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Inference (I)

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/p_fc.csv')

In [5]:
# Adding evidence
model_inference = 'models/ebdt_data/sub_network/p_model/p_model_exp9_inference_200i.pl'

Adding queries

In [6]:
sample_ids = data['p_fc']['sample'].unique().tolist()
phosphosites = data['p_fc']['phosphosite'].unique().tolist()
queries = inf.generate_queries(schema.POccupancyPredicate, sample_ids=sample_ids, phosphosites=phosphosites)
len(queries)

549

In [7]:
queries[:10]

['query(p_occupancy("ABI1(Y213)", "AZD5438", _)).',
 'query(p_occupancy("ANXA2P2(Y24)", "AZD5438", _)).',
 'query(p_occupancy("ATF1(S198)", "AZD5438", _)).',
 'query(p_occupancy("BCLAF1(Y284)", "AZD5438", _)).',
 'query(p_occupancy("HIPK2(Y361)", "AZD5438", _)).',
 'query(p_occupancy("PTTG1IP(Y174)", "AZD5438", _)).',
 'query(p_occupancy("PXN(Y118)", "AZD5438", _)).',
 'query(p_occupancy("PXN(Y88)", "AZD5438", _)).',
 'query(p_occupancy("RBM39(Y95)", "AZD5438", _)).',
 'query(p_occupancy("ABI1(Y213)", "FRAX486", _)).']

Adding evidence (phosphosite and relevant enzymes)

In [8]:
data['es_interaction'] = pd.read_csv('data/processed/ebdt_data/sub_network_p/es_interaction.csv')

In [9]:
# Mapping data to Problog predicates
predicates = {}
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(data['p_fc'], 'phosphosite', 'sample', 'value')
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(data['e_ksea'], 'enzyme', 'sample', 'value')

In [10]:
evidence_dict = {}
for s in sample_ids:
    evidence_dict[s] = {}
    for p in phosphosites:
        enzymes = data['es_interaction']['enzyme'][data['es_interaction']['phosphosite'] == p].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = []
        for e in enzymes:
            evidence_e.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e]))
        evid_generator = build.ProblogStatementGenerator(predicates['p_fc'])
        evidence_p = evid_generator.generate_facts(build.EvidenceTemplate, select=[s, p])
        evidence_dict[s][p] = evidence_e + evidence_p

In [11]:
# make evidence_dict['AZD5438'] into list
evidence_list = []
for s in evidence_dict.keys():
    for p in evidence_dict[s]:
        evidence_list.append(evidence_dict[s][p])
evidence_list[:10]

[['evidence(e_ksea("ABL1", "AZD5438", inc)).',
  'evidence(p_fc("ABI1(Y213)", "AZD5438", inc)).'],
 ['evidence(e_ksea("SRC", "AZD5438", inc)).',
  'evidence(p_fc("ANXA2P2(Y24)", "AZD5438", dec)).'],
 ['evidence(e_ksea("HIPK2", "AZD5438", dec)).',
  'evidence(p_fc("ATF1(S198)", "AZD5438", dec)).'],
 ['evidence(e_ksea("SRC", "AZD5438", inc)).',
  'evidence(p_fc("BCLAF1(Y284)", "AZD5438", inc)).'],
 ['evidence(e_ksea("SRC", "AZD5438", inc)).',
  'evidence(e_ksea("HIPK2", "AZD5438", dec)).',
  'evidence(p_fc("HIPK2(Y361)", "AZD5438", dec)).'],
 ['evidence(e_ksea("SRC", "AZD5438", inc)).',
  'evidence(p_fc("PTTG1IP(Y174)", "AZD5438", dec)).'],
 ['evidence(e_ksea("SRC", "AZD5438", inc)).',
  'evidence(e_ksea("ABL1", "AZD5438", inc)).',
  'evidence(e_ksea("PTK2", "AZD5438", dec)).',
  'evidence(e_ksea("PTPRG", "AZD5438", inc)).',
  'evidence(e_ksea("FYN", "AZD5438", inc)).',
  'evidence(e_ksea("PTK6", "AZD5438", inc)).',
  'evidence(p_fc("PXN(Y118)", "AZD5438", dec)).'],
 ['evidence(e_ksea("S

In [12]:
inference_output = 'models/ebdt_data/sub_network/p_model/inference_log_9.txt'
inf_data_dict = inf.infer_marginf(model_inference, queries, inference_output, evidence=evidence_list)
pickle.dump(inf_data_dict, open('models/ebdt_data/sub_network/p_model/predictions_marginf_relepev_exp9_200i.pkl', 'wb'))

['query(p_occupancy("ABI1(Y213)", "AZD5438", _)).']
1/549
Inferring data (['query(p_occupancy("ABI1(Y213)", "AZD5438", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp9_inference_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_9.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0000s
[INFO] Grounding: 0.0309s
[INFO] Cycle breaking: 0.0036s
[INFO] Clark's completion: 0.0003s
[INFO] DSharp compilation: 0.0597s
[INFO] Total time: 2.3385s
Finished inference (['query(p_occupancy("ABI1(Y213)", "AZD5438", _)).'])...
['query(p_occupancy("ANXA2P2(Y24)", "AZD5438", _)).']
2/549
Inferring data (['query(p_occupancy("ANXA2P2(Y24)", "AZD5438", _)).'])...
problog models/ebdt_data/sub_network/p_model/p_model_exp9_inference_200i.pl -o models/ebdt_data/sub_network/p_model/inference_log_9.txt -k ddnnf -v
[INFO] Output level: INFO
[INFO] Propagating evidence: 0.0000s
[INFO] Grounding: 0.0312s
[INFO] Cycle breaking: 0.0036s
[INFO] Clark's completion: 